In [2]:
import torch
import transformers
model_name_or_path = 'checkpoints/tinyllama_caption2instruct_ft'
model = transformers.LlamaForCausalLM.from_pretrained(
        model_name_or_path,
        attn_implementation='flash_attention_2',
        torch_dtype=torch.bfloat16,
    ).cuda()
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
from instruct_tri2tri.tinyllama_ft.conversation import conv_llama_2
conv = conv_llama_2.copy()
roles = conv_llama_2.roles
inp = '3D rendering of a white sofa with wooden legs and frame.'
conv.append_message(conv_llama_2.roles[0], inp)
conv.append_message(conv_llama_2.roles[1], None)
prompt = conv.get_prompt()
inputs = tokenizer(prompt, return_tensors='pt')
for key, value in inputs.items():
    inputs[key] = value.cuda()

In [2]:
import json

j = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_550k.json'))

In [3]:
j[0]

{'id': '53d0b31aa7f84bc4b1733224963d0114',
 'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
 'conversations': [{'from': 'human',
   'value': '<triplane>\nDescribe the 3D model concisely.'},
  {'from': 'gpt',
   'value': '3D rendering of a white sofa with wooden legs and frame.'}]}

In [18]:
output = model.generate(**inputs)
# tokenizer.decode(output[0], special_tokens=True)

In [24]:
tokenizer.decode(output[0, inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

'make the sofa a brown leather sofa'

In [5]:
import json 
json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_0_4.json'))

[{'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
  'caption': '3D rendering of a white sofa with wooden legs and frame.',
  'instruct': 'make the sofa a leather sofa'},
 {'image': 'objaverse/Cap3D_imgs_view0/25f25e35aada40e49194657fd51c1199',
  'caption': '3D model of a wooden fence with posts and gate, featuring a sand base.',
  'instruct': 'make the fence made of metal'},
 {'image': 'objaverse/Cap3D_imgs_view0/9ff331513cea44a2938d09a03d7b0493',
  'caption': 'White plastic cylinder resembling a toilet paper holder.',
  'instruct': 'make it a toilet paper holder made of gold'},
 {'image': 'objaverse/Cap3D_imgs_view0/f78e38ccea1d45dcaefe1970e7df4035',
  'caption': '3D model of a forklift truck with a trailer.',
  'instruct': 'make the truck a tank'},
 {'image': 'objaverse/Cap3D_imgs_view0/112c059282cf4511a01fd27211edcae8',
  'caption': 'A 3D model of a wooden table and bench.',
  'instruct': 'make it a wooden table and bench made of marble'},
 {'image': 'objaver

In [ ]:
from instruct_tri2tri.tsr.system import TSR, InstructTri2Tri
model = InstructTri2Tri.from_pretrained(
    'instruct_tri2tri/instruct_tri2tri_config',
    config_name="config.yaml",
    weight_name="model.ckpt",
)

In [ ]:
import numpy as np
from PIL import Image
image = Image.fromarray(np.zeros((512, 512, 3), dtype=np.uint8))

In [ ]:
tokens = model([image], 'turn it to red', 'cpu', False)
tokens.shape

In [ ]:
scene_codes = model([image], 'turn it to red', 'cpu', True)
scene_codes.shape

In [11]:
import os
import json

data0 = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_0_4.json'))
data1 = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_1_4.json'))
data2 = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_2_4.json'))
data3 = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_3_4.json'))

In [12]:
data = data0 + data1 + data2 + data3
len(data)

493857

In [14]:
data[0]

{'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
 'caption': '3D rendering of a white sofa with wooden legs and frame.',
 'instruct': 'make the sofa a leather sofa'}

In [13]:
with open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k.json', 'w') as f:
    json.dump(data, f)

In [15]:
from PIL import Image
from tqdm import tqdm
import numpy as np
import rembg
from instruct_tri2tri.tsr.utils import remove_background, resize_foreground

save_path = 'data/objaverse/images'
img_names = os.listdir('data/objaverse/Cap3D_imgs_view0')
os.makedirs(save_path, exist_ok=True)
rembg_session = rembg.new_session()
for img_name in tqdm(img_names):
    image = Image.open(os.path.join('data/objaverse/Cap3D_imgs_view0', img_name))
    image = remove_background(image, rembg_session)
    image = resize_foreground(image, 0.85)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[:, :, :3] * image[:, :, 3:4] + (1 - image[:, :, 3:4]) * 0.5
    image = Image.fromarray((image * 255.0).astype(np.uint8))
    image.save(f'{save_path}/{img_name}.jpg')

  0%|          | 41/661577 [00:15<69:07:46,  2.66it/s] 


KeyboardInterrupt: 

In [5]:
from instruct_tri2tri.tsr.system import InstructTri2Tri

model = InstructTri2Tri.from_pretrained(
    'instruct_tri2tri/tsr/instruct_tri2tri_config',
    'config.yaml',
    'model.ckpt'
)

/home/shangyu/Cyberpunk8T/anaconda_space/envs/langsplat/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [6]:
model.instruction_converter.load_state_dict(model.backbone.state_dict())

<All keys matched successfully>

In [3]:
import numpy as np
from PIL import Image

image0 = Image.fromarray(np.zeros((512, 512, 3), dtype=np.uint8))
image1 = Image.fromarray(np.zeros((512, 512, 3), dtype=np.uint8))

In [7]:
token0 = model.forward_tsr([image0], 'cpu', False)
token0.shape

torch.Size([1, 1024, 3072])

In [8]:
token1 = model([image1], 'make it green', 'cpu', False)
token1.shape

torch.Size([1, 1024, 3072])